
# **Install requirements**

In [1]:
!pip3 install torch torchvision torchaudio

In [2]:
!pip3 install 'tqdm' icecream

# **Import libraries**

In [3]:
import os
import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from torchvision.models import alexnet, AlexNet_Weights

from PIL import Image
from tqdm.notebook import tqdm
from pprint import pprint, pformat
from icecream import ic
import shelve

# **Set Arguments**

In [4]:
DEVICE = 'cpu' # 'cuda' or 'cpu'

NUM_CLASSES = 101 # 101 + 1: There is an extra Background class that should be removed

BATCH_SIZE = 256     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 1e-2            # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default

NUM_EPOCHS = 30      # Total number of training epochs (iterations over dataset)
STEP_SIZE = 10       # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 16

# **Define Data Preprocessing**

In [5]:
# Define transforms for training phase
mean_center, std_center = [0.5,0.5,0.5],[0.5,0.5,0.5],
mean_101, std_101 = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225] # src: https://pytorch.org/vision/stable/models/generated/torchvision.models.alexnet.html#torchvision.models.AlexNet_Weights
mean, std = mean_101, std_101
train_transform = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ColorJitter(brightness=.1) ,
                                      transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize(mean, std) # Normalizes tensor with mean and standard deviation
])
# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean, std)
])

# **Prepare Dataset**

In [6]:
from torchvision.datasets import VisionDataset

from PIL import Image
from typing import Union, Literal

import os
import os.path
import sys


def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')


class Caltech(VisionDataset):
    def __init__(self, root, split: Literal['train', 'test']='train', transform=None, target_transform=None):
        super(Caltech, self).__init__(root, transform=transform, target_transform=target_transform)

        if not os.path.isdir(self.root):
          raise FileNotFoundError(f"{self.root} not found")

        FILTER_OUT = "BACKGROUND"

        self.labels = sorted([f.split("/")[0] for f in os.listdir(self.root) if FILTER_OUT not in f])

        self.split = split # This defines the split you are going to use
                           # (split files are called 'train.txt' and 'test.txt')
        self.filepath = os.path.join(self.root, "..", f"{self.split}.txt")
        if not os.path.isfile(self.filepath):
          raise FileNotFoundError(f"{self.filepath} not found")



        with open(self.filepath, "r") as f:
          self.files = [f.strip() for f in f.readlines() if FILTER_OUT not in f]

        '''
        - Here you should implement the logic for reading the splits files and accessing elements
        - If the RAM size allows it, it is faster to store all data in memory
        - PyTorch Dataset classes use indexes to read elements
        - You should provide a way for the __getitem__ method to access the image-label pair
          through the index
        - Labels should start from 0, so for Caltech you will have lables 0...100 (excluding the background class)
        '''

    def __getitem__(self, index: int):
        '''
        __getitem__ should access an element through its index
        Args:
            index (int): Index

        Returns:
            tuple: (sample, target) where target is class_index of the target class.
        '''

        filename = self.files[index].strip()
        label_name = filename.split("/")[0]
        label = self.labels.index(label_name)
        imagepath = os.path.join(self.root, filename)
        image = pil_loader(imagepath)
        # Applies preprocessing when accessing the image
        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def __len__(self):
        '''
        The __len__ method returns the length of the dataset
        It is mandatory, as this is used by several other components
        '''
        length = len(self.files) # Provide a way to get the length (number of elements) of the dataset
        return length

In [7]:
# Clone github repository with data
import pathlib
from sklearn.model_selection import train_test_split
print(pathlib.Path().resolve())
if not os.path.isdir('./Caltech101'):
  !git clone https://github.com/MachineLearning2020/Homework2-Caltech101.git
  !mv 'Homework2-Caltech101' 'Caltech101'

DATA_DIR = 'Caltech101/101_ObjectCategories'

# Prepare Pytorch train/test Datasets
train_dataset = Caltech(DATA_DIR, split='train',  transform=train_transform)
test_dataset = Caltech(DATA_DIR, split='test', transform=eval_transform)
ic(len(train_dataset.labels))
train_indexes, val_indexes = train_test_split(list(range(len(train_dataset))),
                                              random_state=42,
                                              stratify=[fn.split("/")[0] for fn in train_dataset.files],
                                              test_size=0.25)

val_dataset = Subset(train_dataset, val_indexes)
train_dataset = Subset(train_dataset, train_indexes)

# Check dataset sizes
print('Train Dataset: {}'.format(len(train_dataset)))
print('Valid Dataset: {}'.format(len(val_dataset)))
print('Test Dataset: {}'.format(len(test_dataset)))

/content
Cloning into 'Homework2-Caltech101'...
remote: Enumerating objects: 9256, done.
remote: Total 9256 (delta 0), reused 0 (delta 0), pack-reused 9256
Receiving objects: 100% (9256/9256), 129.48 MiB | 20.60 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Updating files: 100% (9149/9149), done.


ic| len(train_dataset.labels): 101


Train Dataset: 4338
Valid Dataset: 1446
Test Dataset: 2893


# **Prepare Dataloaders**

In [8]:
# Dataloaders iterate over pytorch datasets and transparently provide useful functions (e.g. parallelization and shuffling)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
pprint((len(train_dataloader.dataset.dataset), len(train_dataloader.dataset.indices)))
pprint((len(val_dataloader.dataset.dataset), len(val_dataloader.dataset.indices)))

(5784, 4338)
(5784, 1446)


# **Prepare Network**

In [9]:
net = alexnet(weights=AlexNet_Weights.IMAGENET1K_V1) # Loading AlexNet model

# AlexNet has 1000 output neurons, corresponding to the 1000 ImageNet's classes
# We need 101 outputs for Caltech-101
net.classifier[6] = nn.Linear(4096, NUM_CLASSES) # nn.Linear in pytorch is a fully connected layer
                                                 # The convolutional layer is nn.Conv2d

# We just changed the last layer of AlexNet with a new fully connected layer with 101 outputs
# It is strongly suggested to study torchvision.models.alexnet source code

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:03<00:00, 65.4MB/s]


# **Prepare Training**

In [10]:
# Define loss function
criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

# Choose parameters to optimize
# To access a different set of parameters, you have to access submodules of AlexNet
# (nn.Module objects, like AlexNet, implement the Composite Pattern)
# e.g.: parameters of the fully connected layers: net.classifier.parameters()
# e.g.: parameters of the convolutional layers: look at alexnet's source code ;)
parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet
# parameters_to_optimize = net.features.parameters() # In this case we optimize over all the convolution layers' parameters of AlexNet
# parameters_to_optimize = net.classifier.parameters() # In this case we optimize over all the fully connected layers' parameters of AlexNet

# Define optimizer
# An optimizer updates the weights based on loss
# We use SGD with momentum
optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

# Define scheduler
# A scheduler dynamically changes learning rate
# The most common schedule is the step(-down), which multiplies learning rate by gamma every STEP_SIZE epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)
print(len(train_dataloader))

16


# **Train**

# **Validation**

In [11]:
def _validate(net, val_dataloader, val_dataset):
  net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
  net.train(False) # Set Network to evaluation mode

  running_corrects = 0
  for images, labels in tqdm(val_dataloader, unit="step", leave=False, desc="Validation"):
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    # Forward Pass
    outputs = net(images)

    # Get predictions
    _, preds = torch.max(outputs.data, 1)

    # Update Corrects
    running_corrects += torch.sum(preds == labels.data).data.item()

  # Calculate Accuracy
  accuracy = running_corrects / float(len(val_dataset))
  net.train()

  print('Validation Accuracy: {:.2%}'.format(accuracy))
  return accuracy
validate = lambda: _validate(net, val_dataloader, val_dataset)

In [ ]:
# By default, everything is loaded to cpu
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

cudnn.benchmark # Calling this optimizes runtime

current_step = 0
current_loss = None
# Start iterating over the epochs
pbar = tqdm(range(NUM_EPOCHS), unit="epoch")
best_so_far = {
    "accuracy": None,
    "epoch": None,
    "model": None,
    "loss": None
}
for epoch in pbar:
  lr = scheduler.get_last_lr()
  pbar.set_description(f"Training - LR={lr}, Step={current_step}, Loss={(current_loss if current_loss is not None else 1):.2%}")
  # print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, lr))

  # Iterate over the dataset
  for images, labels in tqdm(train_dataloader, leave=False, unit="step"):
    # Bring data over the device of choice
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    net.train() # Sets module in training mode

    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
    optimizer.zero_grad() # Zero-ing the gradients

    # Forward pass to the network
    outputs = net(images)

    # Compute loss based on output and ground truth
    loss = criterion(outputs, labels)

    # Log loss
    # if current_step % LOG_FREQUENCY == 0:
    #   print('Step {}, Loss {}'.format(current_step, loss.item()))

    current_step += 1
    current_loss = loss.item()

    # Compute gradients for each layer and update weights
    loss.backward()  # backward pass: computes gradients
    optimizer.step() # update weights based on accumulated gradients

  acc = validate()

  if best_so_far['accuracy'] is None or acc > best_so_far['accuracy']:
    best_so_far = {
        "accuracy": acc,
        "epoch": epoch,
        "model": net.state_dict(),
        "loss": current_loss
    }


  # Step the scheduler
  scheduler.step()

# **Test**

In [ ]:
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode
net.load_state_dict(best_so_far["model"])
running_corrects = 0
for images, labels in tqdm(test_dataloader, unit="step", desc=f"Testing with {best_so_far['epoch']+1}-epoch's model"):
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

  # Forward Pass
  outputs = net(images)

  # Get predictions
  _, preds = torch.max(outputs.data, 1)

  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset))

print('Test Accuracy: {}'.format(accuracy))

In [ ]:
def persist(name):
  global accuracy, LR, best_so_far, GAMMA, STEP_SIZE, NUM_EPOCHS
  with shelve.open("backup") as backup:
    backup[name] = {
        'LR': LR,
        "STEP_SIZE": STEP_SIZE,
        "NUM_EPOCHS": NUM_EPOCHS,
        "GAMMA": GAMMA,
        "model": best_so_far,
        'accuracy': accuracy
    }
    backup.close()
  persist("PRE_TRAINING_101_FREEZE_CONV_DATA_AUG_HF_ALLPARAM")